In [1]:
from docx import Document
import pandas as pd
from faker import Faker
from docx.shared import Cm
import random

In [2]:
def generate_fake_details(count):
    fake = Faker()

    # generate n fake records
    records = []
    for _ in range(count):
        doctor_name = f"{fake.first_name()}, {fake.last_name()}"
        doctor_lastname = doctor_name.split(', ')[1].strip()
        doctor_address = fake.address().replace("\n", ", ")
        doctor_phone = fake.numerify(text="##########")
        doctor_fax = fake.numerify(text="##########")
        registration_date = fake.date_between(start_date="-10y", end_date="today").strftime("%Y-%m-%d")

        patient_name = f"{fake.first_name_female()}, {fake.last_name_female()}"
        patient_insurance = fake.numerify(text="##########")
        patient_mobile = fake.numerify(text="##########")
        patient_dob = fake.date_of_birth(minimum_age=18, maximum_age=90).strftime("%d-%b-%Y")
        patient_address = fake.address().replace("\n", ", ")
        expected_due_date = fake.date_between(start_date="today", end_date="+9M").strftime("%d-%b-%Y")

        records.append([
            doctor_name, doctor_lastname,registration_date, doctor_address, doctor_phone, doctor_fax,
            patient_name, patient_insurance, patient_dob, patient_address, patient_mobile, expected_due_date
        ])

    # create and save dataFrame
    columns = [
        "doctor_name", "doctor_lastname","date_of_registration", "doctor_address", "doctor_phone",
        "doctor_fax", "patient_name", "patient_insurance_number", "patient_dob", "patient_address",
        "patient_mobile_number", "expected_due_date"
    ]
    df = pd.DataFrame(records, columns=columns)
    filepath = "fake_patient_contacts.csv"
    df.to_csv(filepath, index=False)
    print("CSV file generated successfully!")
    
    return filepath

In [3]:
def fill_due_date_info(template_path,output_path, data):
    
    doc = Document(template_path)
    
    # replace placeholder's value
    for paragraph in doc.paragraphs:
        
        if "<img>" in paragraph.text:
            paragraph.text = paragraph.text.replace("<img>", "")
            run = paragraph.add_run()
            run.add_picture(f'gynae logos/{random.randint(1, 8)}.png', width=Cm(5))
        elif "<signature>" in paragraph.text:
            paragraph.text = paragraph.text.replace("<signature>", "")
            run = paragraph.add_run()
            run.add_picture(f'signatures/{random.randint(1, 8)}.png', width=Cm(2))
        else:
            for key, value in data.items():
                if key in paragraph.text:
                    for run in paragraph.runs:
                        run.text = run.text.replace(key, str(value))
    
    doc.save(output_path)
    
def generate_bill_from_csv(csv_path,template_path):
    
    df = pd.read_csv(csv_path)
    
    # generate data for replacement from csv
    for idx, row in df.iterrows():
        patient_first_name = row['patient_name'].split(',')[0].strip()
        data={
            '[Doctor name]' : row['doctor_name'],
            '[Date of registration]' : row['date_of_registration'],
            '[Doctor Lastname]' : row['doctor_lastname'],
            '[Doctor address]': row['doctor_address'],
            '[Doctor phone]' : row['doctor_phone'],
            '[Doctor fax]' : row['doctor_fax'],
            '[Patient name]' : row['patient_name'],
            '[Patient insurance number]': row['patient_insurance_number'],
            '[Patient DOB]': row['patient_dob'],
            '[Patient address]' : row['patient_address'],
            '[Patient mobile number]': row['patient_mobile_number'],
            '[expected due date]' : row['expected_due_date']
        }
    
    
        output_path = f'Pregnancy Bill/{patient_first_name}_duedate.docx'    
        fill_due_date_info(template_path,output_path,data)
        
    print("Filled medical bills successfully!")

In [4]:
if __name__ == "__main__":
    template_path = 'Bill Template.docx'
    csv_path = generate_fake_details(100)
    generate_bill_from_csv(csv_path,template_path)

CSV file generated successfully!
Filled medical bills successfully!
